In [152]:
import pandas as pd
import numpy as np


# source of the function - kaggle itself- -path taken from metadata
import kagglehub
from kagglehub import KaggleDatasetAdapter

# EDA

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

import joblib

# Overall Approach


*   Data curation and feature engineerinng
*   model building fir top5 products based on sales
-- model building happened using addition of incremental features like week of the year and day of the week, 2 diffrent models tested Randomforest and Gradient boosting
-- here the sales distribution at individual level for any product for top 5 products by sales is less - mostly either 1 or 2 products being sold  owing to which model would have limited pattern to train on

- - Key Assumption: to make model building framework which is scalleble and deploy it in pipeline. further model tuning is not explored






# Data extraction using kaggle

In [22]:


# Set the path to the file you'd like to load
# file_path = "Sales_data/Sales_April_2019.csv"
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
file_path_list = [f"Sales_data/Sales_{i}_2019.csv" for i in months ]
df = pd.DataFrame()
for i in file_path_list:
  # print(i)
  # Load the latest version
  df_i = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "pigment/big-sales-data",
  i,
  )
  df = pd.concat([df, df_i],ignore_index=True)




<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_i = kagglehub.load_dataset(
<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_i = kagglehub.load_dataset(
<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_i = kagglehub.load_dataset(
<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_i = kagglehub.load_dataset(
<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_i = kagglehub.load_dataset(
<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_i = kagglehub.load_dataset(
<ipython-input-22-062718ef17ea>:9: DeprecationWarning: load_dataset is depre

In [23]:
df.shape

(186850, 6)

In [24]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [103]:
# df.to_csv('sales_data_year2019.csv') # storing data for multiple usage

In [5]:
cd

/root


In [25]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [26]:
df.shape

(186850, 6)

# Data cleaning and exploration

In [27]:
df.isna().sum()

,0
Order ID,545
Product,545
Quantity Ordered,545
Price Each,545
Order Date,545
Purchase Address,545


In [28]:
df = df.dropna()
# only 545 ~<0.1 % nulls - dropping them

In [29]:
df.shape

(186305, 6)

In [30]:
df.head(100)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
...,...,...,...,...,...,...
95,141327,USB-C Charging Cable,1,11.95,01/31/19 01:38,"238 Washington St, Los Angeles, CA 90001"
96,141328,ThinkPad Laptop,1,999.99,01/06/19 23:18,"736 5th St, Seattle, WA 98101"
97,141329,USB-C Charging Cable,1,11.95,01/01/19 16:01,"122 5th St, Portland, ME 04101"
98,141330,Wired Headphones,1,11.99,01/21/19 13:04,"391 5th St, Portland, OR 97035"


# Feature engineering

In [31]:
# plan is to see sales of products across addresses
df['postal code'] = df['Purchase Address'].str[-8:]

In [33]:
df = df[df['Quantity Ordered']!= 'Quantity Ordered']

In [34]:
df.head(1070)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,postal code
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215",MA 02215
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035",OR 97035
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016",CA 94016
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001",CA 90001
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301",TX 73301
...,...,...,...,...,...,...,...
1069,142256,AAA Batteries (4-pack),1,2.99,01/25/19 13:37,"226 Jefferson St, San Francisco, CA 94016",CA 94016
1070,142257,iPhone,1,700,01/01/19 12:16,"774 2nd St, Atlanta, GA 30301",GA 30301
1071,142258,Lightning Charging Cable,1,14.95,01/05/19 23:40,"251 Lakeview St, Atlanta, GA 30301",GA 30301
1072,142259,LG Dryer,1,600.0,01/20/19 08:03,"413 Sunset St, New York City, NY 10001",NY 10001


In [35]:
# have to find months out of dates for the insights
df['Month'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M').dt.month_name()

In [129]:
# have to find months out of dates for the insights
# adding this feature after second run
# adding weekof the year as a final feture from date

df['dayofweek'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M').dt.dayofweek
df['week'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M').dt.isocalendar().week

In [130]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,postal code,Month,City,sales,dayofweek,week
0,141234,iPhone,1,700.00,01/22/19 21:25,"944 Walnut St, Boston, MA 02215",MA 02215,January,Boston,700.00,1,4
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035",OR 97035,January,Portland,14.95,0,5
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016",CA 94016,January,San Francisco,23.98,3,3
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001",CA 90001,January,Los Angeles,149.99,5,1
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301",TX 73301,January,Austin,11.99,4,4


In [36]:
# Extract City from Address
df['City'] = df['Purchase Address'].apply(lambda x: x.split(',')[1].strip())

In [131]:
df['Price Each'] = df['Price Each'].astype(float)
df.nunique()

,0
Order ID,178437
Product,19
Quantity Ordered,9
Price Each,17
Order Date,142395
Purchase Address,140787
postal code,10
Month,12
City,9
sales,54


In [132]:
# need to create sales column
df['sales'] = df['Quantity Ordered'].astype(float) * df['Price Each'].astype(float)

In [133]:
product_sales = df.groupby('Product').agg(
    Total_Sales=('sales', 'sum'),
    Unique_Price=('Price Each', 'mean')
).reset_index()
product_sales.sort_values(by='Total_Sales', ascending=False, inplace=True)

In [134]:
product_sales.head()

,Product,Total_Sales,Unique_Price
13,Macbook Pro Laptop,8037600.00,1700.00
18,iPhone,4794300.00,700.00
14,ThinkPad Laptop,4129958.70,999.99
9,Google Phone,3319200.00,600.00
1,27in 4K Gaming Monitor,2435097.56,389.99


In [135]:
# selecting top 5 products by sales
top_5_products = product_sales['Product'].head(5).to_list()

In [136]:
df_product = pd.DataFrame()
df_product['Product'] = top_5_products

In [137]:
df_product.head()

,Product
0,Macbook Pro Laptop
1,iPhone
2,ThinkPad Laptop
3,Google Phone
4,27in 4K Gaming Monitor


In [138]:
# since price is same for all the products ,   quantities ordered is the reference for sales , we are seeing the distribution
df_product = pd.DataFrame()
df_product['Product'] = top_5_products
price_dist = []
quantity_list = []
for product in top_5_products:
    print(f"unique prices for {product}  -",df[df['Product'] == product]['Price Each'].unique())
    print(f"unique quantities for {product}  -",df[df['Product'] == product]['Quantity Ordered'].unique())
    price_dist.append(df[df['Product'] == product]['Price Each'].unique())
    quantity_list.append(list(df[df['Product'] == product]['Quantity Ordered'].unique()))
df_product['price_distribution'] = price_dist
df_product['quantity distribution'] = quantity_list
df_product

unique prices for Macbook Pro Laptop  - [1700.]
unique quantities for Macbook Pro Laptop  - ['1' '2']
unique prices for iPhone  - [700.]
unique quantities for iPhone  - ['1' '2']
unique prices for ThinkPad Laptop  - [999.99]
unique quantities for ThinkPad Laptop  - ['1' '2']
unique prices for Google Phone  - [600.]
unique quantities for Google Phone  - ['1' '2']
unique prices for 27in 4K Gaming Monitor  - [389.99]
unique quantities for 27in 4K Gaming Monitor  - ['1' '2']


,Product,price_distribution,quantity distribution
0,Macbook Pro Laptop,[1700.0],"[1, 2]"
1,iPhone,[700.0],"[1, 2]"
2,ThinkPad Laptop,[999.99],"[1, 2]"
3,Google Phone,[600.0],"[1, 2]"
4,27in 4K Gaming Monitor,[389.99],"[1, 2]"


In [139]:
top_5_products

['Macbook Pro Laptop',
 'iPhone',
 'ThinkPad Laptop',
 'Google Phone',
 '27in 4K Gaming Monitor']

In [140]:
# in data for individual products
# - the price is remains same also the sales would be quantity indicator so we wont be using those features,
# also address is converted to postal codes so removing those as well
#  City is redudnant as postal code is more granual and  ~highly corelated to postal code
# given the price is same 'Quantity Ordered' is direct representative of sales so removing those

In [141]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address', 'postal code', 'Month', 'City', 'sales',
       'dayofweek', 'week'],
      dtype='object')

In [142]:
df_model = df.drop(columns=['Purchase Address','Order Date','Order ID','Price Each','City','Quantity Ordered' ])

In [143]:
df_model['postal code'] = df_model['postal code'].str.replace(' ', '_')

In [144]:
# encoding postal code and month columns

In [145]:
df_model_encoded = pd.get_dummies(df_model, columns=['postal code', 'Month'],dtype=int)

In [146]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address', 'postal code', 'Month', 'City', 'sales',
       'dayofweek', 'week'],
      dtype='object')

In [147]:
df_model_encoded.head()

,Product,sales,dayofweek,week,postal code_CA_90001,postal code_CA_94016,postal code_GA_30301,postal code_MA_02215,postal code_ME_04101,postal code_NY_10001,...,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September
0,iPhone,700.00,1,4,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Lightning Charging Cable,14.95,0,5,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Wired Headphones,23.98,3,3,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,27in FHD Monitor,149.99,5,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Wired Headphones,11.99,4,4,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [148]:
df_model_encoded.columns

Index(['Product', 'sales', 'dayofweek', 'week', 'postal code_CA_90001',
       'postal code_CA_94016', 'postal code_GA_30301', 'postal code_MA_02215',
       'postal code_ME_04101', 'postal code_NY_10001', 'postal code_OR_97035',
       'postal code_TX_73301', 'postal code_TX_75001', 'postal code_WA_98101',
       'Month_April', 'Month_August', 'Month_December', 'Month_February',
       'Month_January', 'Month_July', 'Month_June', 'Month_March', 'Month_May',
       'Month_November', 'Month_October', 'Month_September'],
      dtype='object')

# Model building and validation

In [149]:
model_dict = {}
for product in top_5_products:
    df_model_data = df_model_encoded[df_model_encoded['Product'] == product].drop(columns=['Product'])
    X = df_model_data.drop(columns=['sales'])
    y = df_model_data['sales']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training model for {product} with training data {len(X_train)}")
    print(f"diffrent sales amount in the data {y_test.unique()}")
    print(f"Test data length {len(X_test)}")
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    model_dict[product] = model # storing the models for further usage in the notebook
    joblib.dump(model, f"{product}_RF_model.pkl")
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Squared Error for {product}: {mse}")
    print(f"R-squared for {product}: {r2}")





Training model for Macbook Pro Laptop with training data 3779
diffrent sales amount in the data [1700.]
Test data length 945
Mean Squared Error for Macbook Pro Laptop: 275.3264328666213
R-squared for Macbook Pro Laptop: 0.0
Training model for iPhone with training data 5473
diffrent sales amount in the data [ 700. 1400.]
Test data length 1369
Mean Squared Error for iPhone: 1045.3151548169792
R-squared for iPhone: -0.4623776897622862
Training model for ThinkPad Laptop with training data 3302
diffrent sales amount in the data [999.99]
Test data length 826
Mean Squared Error for ThinkPad Laptop: 634.7902666743845
R-squared for ThinkPad Laptop: 0.0
Training model for Google Phone with training data 4420
diffrent sales amount in the data [600.]
Test data length 1105
Mean Squared Error for Google Phone: 118.1208361989
R-squared for Google Phone: 0.0
Training model for 27in 4K Gaming Monitor with training data 4984
diffrent sales amount in the data [389.99 779.98]
Test data length 1246
Mean Sq

### Gradientboosting algorythm fitting -

In [153]:
# iterating gradientboosting - GradientBoostingRegressor
model_dict = {}
for product in top_5_products:
    df_model_data = df_model_encoded[df_model_encoded['Product'] == product].drop(columns=['Product'])
    X = df_model_data.drop(columns=['sales'])
    y = df_model_data['sales']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training model for {product} with training data {len(X_train)}")
    print(f"diffrent sales amount in the data {y_test.unique()}")
    print(f"Test data length {len(X_test)}")
    model = GradientBoostingRegressor(n_estimators=500, random_state=42)
    model.fit(X_train, y_train)
    model_dict[product] = model # storing the models for further usage in the notebook
    joblib.dump(model, f"{product}_gb_model.pkl")
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Squared Error for {product}: {mse}")
    print(f"R-squared for {product}: {r2}")


Training model for Macbook Pro Laptop with training data 3779
diffrent sales amount in the data [1700.]
Test data length 945
Mean Squared Error for Macbook Pro Laptop: 420.01274580398854
R-squared for Macbook Pro Laptop: 0.0
Training model for iPhone with training data 5473
diffrent sales amount in the data [ 700. 1400.]
Test data length 1369
Mean Squared Error for iPhone: 793.6900438478306
R-squared for iPhone: -0.11035854341242834
Training model for ThinkPad Laptop with training data 3302
diffrent sales amount in the data [999.99]
Test data length 826
Mean Squared Error for ThinkPad Laptop: 832.0122529379886
R-squared for ThinkPad Laptop: 0.0
Training model for Google Phone with training data 4420
diffrent sales amount in the data [600.]
Test data length 1105
Mean Squared Error for Google Phone: 65.36713407588967
R-squared for Google Phone: 0.0
Training model for 27in 4K Gaming Monitor with training data 4984
diffrent sales amount in the data [389.99 779.98]
Test data length 1246
Mea

In [81]:
# since the quanitites sold is less either 1 or 2 and sales in $ becomes less automatically given the price is same
# mse values are within acceptable and
  # since there is less pattern to learn owing to sales distribution shown bellow, I am not further fine tunning the model

In [ ]:
# since price is same for all the products ,   quantities ordered is the reference for sales , we are seeing the distribution
df_product = pd.DataFrame()
df_product['Product'] = top_5_products
price_dist = []
quantity_list = []
for product in top_5_products:
    print(f"unique prices for {product}  -",df[df['Product'] == product]['Price Each'].unique())
    print(f"unique quantities for {product}  -",df[df['Product'] == product]['Quantity Ordered'].unique())
    price_dist.append(df[df['Product'] == product]['Price Each'].unique())
    quantity_list.append(list(df[df['Product'] == product]['Quantity Ordered'].unique()))
df_product['price_distribution'] = price_dist
df_product['quantity distribution'] = quantity_list
df_product

unique prices for Macbook Pro Laptop  - [1700.]
unique quantities for Macbook Pro Laptop  - ['1' '2']
unique prices for iPhone  - [700.]
unique quantities for iPhone  - ['1' '2']
unique prices for ThinkPad Laptop  - [999.99]
unique quantities for ThinkPad Laptop  - ['1' '2']
unique prices for Google Phone  - [600.]
unique quantities for Google Phone  - ['1' '2']
unique prices for 27in 4K Gaming Monitor  - [389.99]
unique quantities for 27in 4K Gaming Monitor  - ['1' '2']


,Product,price_distribution,quantity distribution
0,Macbook Pro Laptop,[1700.0],"[1, 2]"
1,iPhone,[700.0],"[1, 2]"
2,ThinkPad Laptop,[999.99],"[1, 2]"
3,Google Phone,[600.0],"[1, 2]"
4,27in 4K Gaming Monitor,[389.99],"[1, 2]"
